<a href="https://colab.research.google.com/github/1975JHK/1975JHK.github.io/blob/main/performance_with_Bagging_and_OOD_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Performance Comparison --------------------------------------------------
## 4. Bagging + Baseline OOD Detection
options(warn=-1)

new.result <- data.frame(trail.1 = rep(NA, times = nrow(test)))
for(i in 1:101){
  trial <- c(rep(NA, times = nrow(test)))
  new.result <- data.frame(new.result, trial)}

new.result2 <- cbind(
  new.result[ , -c(1:2)],
  majority = rep(NA, times = nrow(test)),
  count = rep(NA, times = nrow(test)),
  rate = rep(NA, times = nrow(test))
)

for(i in 1:100){
  index <- sample(1:nrow(train), nrow(train)*0.80, replace = F)
  new.train <- train[index, ]
  new.svm <- svm(Y3 ~ ., data = new.train[ , c(1:25, 27)],
                 type = type,
                 kernel = best2[1, 1],
                 cost = best2[1, 2],
                 gamma = best2[1, 3],
                 epsilon = best2[1, 4],
                 class.weights = list('Normal' = 1.0, 'Good' = 1.5, 'Bad' = 2.0))
  new.pred = predict(new.svm, newdata = test[ , c(1:25)])
  new.result2[ , i] <- new.pred
}

new.result2$majority <- apply(new.result2[ , 1:100], 1, function(x) names(which.max(table(x))))
new.result2$count <- apply(new.result2[ , 1:100], 1, function(x) max(table(x)))
new.result2$rate <- new.result2$count/100

finals_sh <- data.frame(new.result2[ , c(101:103)], test$Y3)
test_kit2 <- data.frame(rate = seq(0.800, 0.999, by = 0.001),
                       accuraty = rep(NA, length = 200))
for(i in 1:nrow(test_kit2)){
  finals_sh2 <- finals_sh %>% filter(test_kit2[i, 1] < rate)
  cm = confusionMatrix(as.factor(finals_sh2$majority), 
                       finals_sh2$test.Y3)
  test_kit2[i, 2] = cm$overall[1]
}

best_rate2 <- test_kit2[which.max(test_kit2[ , 2]), 1]
finals_sh <- finals_sh %>% filter(best_rate2[1]+0.005 <= rate)

new.cm4 = confusionMatrix(as.factor(finals_sh$majority), finals_sh$test.Y3)
print(new.cm4)


Confusion Matrix and Statistics

          Reference
Prediction Normal Good Bad
    Normal     45    7   0
    Good       27   41   0
    Bad         3    0  33

Overall Statistics
                                          
               Accuracy : 0.7628          
                 95% CI : (0.6882, 0.8272)
    No Information Rate : 0.4808          
    P-Value [Acc > NIR] : 5.635e-13       
                                          
                  Kappa : 0.6389          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: Normal Class: Good Class: Bad
Sensitivity                 0.6000      0.8542     1.0000
Specificity                 0.9136      0.7500     0.9756
Pos Pred Value              0.8654      0.6029     0.9167
Neg Pred Value              0.7115      0.9205     1.0000
Prevalence                  0.4808      0.3077     0.2115
Detection Rate              0.2885      0.2628     0.2115
Det